In [14]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

# Based on Jäckel (2015)
from py_vollib.black_scholes import black_scholes
from py_vollib.black import black

### Import and Preprocess Data

In [ ]:
with open("mapping and calibration/files/calibrated_chunks_hagan.pkl", "rb") as f:
    chunks_hagan = pickle.load(f)
with open("mapping and calibration/files/calibrated_chunks_antonov.pkl", "rb") as f:
    chunks_antonov = pickle.load(f)
with open("mapping and calibration/files/calibrated_chunks_nn.pkl", "rb") as f:
    chunks_nn = pickle.load(f)

In [16]:
# Combine chunks
chunks = []
for df1, df2, df3 in zip(chunks_hagan, chunks_antonov, chunks_nn):
    combined_df = pd.concat([df1, df2, df3], axis=1)
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
    chunks.append(combined_df)

### Compute Arbitrage Violations

In [17]:
# Add Black model call prices using py_vollib
for method in ["nn", "hagan", "antonov"]:
    for i, chunk in enumerate(chunks):
        chunk = chunk.copy()
        chunk[f"call_price_{method}"] = chunk.apply(
            lambda row: black(
                F=float(row["forward_price"]),
                K=float(row["strike_price"]),
                sigma=float(row[f"impl_volatility_{method}"]),
                flag="c",
                r=0.0,
                t=float(row["T"]),
            ),
            axis=1
        )
        chunks[i] = chunk

### Butterfly Spread

In [18]:
# Check for violations of butterfly spread condition
def add_convexity_violation_column(chunks, column_name, tol=0.0000):
    updated_chunks = []

    for chunk in chunks:
        K = chunk["strike_price"].values
        C = chunk[f"{column_name}"].values
        f = chunk["forward_price"].values

        violation_flags = np.zeros(len(chunk), dtype=bool)

        # All strikes have the same distance between them
        for i in range(1, len(C) - 1):
            d1 = (C[i] - C[i - 1]) 
            d2 = (C[i + 1] - C[i]) + tol
            if d2 <= d1:
                violation_flags[i] = True

        if ((C[0] - f[0]) / (K[0] - 0)) >= ((C[1] - C[0] + tol) / (K[1] - K[0])):
            violation_flags[0] = True
        
        chunk[f"{column_name}_butterfly_violation"] = violation_flags
        updated_chunks.append(chunk)

    return updated_chunks

for column_name in ["price", "call_price_nn", "call_price_hagan", "call_price_antonov"]:
    chunks = add_convexity_violation_column(chunks, column_name)


### Positivity

In [19]:
# Check for violations of positivity condition
def add_positivity_violation_column(chunks, column_name):
    updated_chunks = []

    for chunk in chunks:
        C = chunk[f"{column_name}"].values

        violation_flags = np.zeros(len(chunk), dtype=bool)

        for i in range(0, len(C) - 1):
            if C[i] < 0:
                violation_flags[i] = True
        
        chunk[f"{column_name}_positivity_violation"] = violation_flags
        updated_chunks.append(chunk)

    return updated_chunks

for column_name in ["price", "call_price_nn", "call_price_hagan", "call_price_antonov"]:
    chunks = add_positivity_violation_column(chunks, column_name)


### Vertical Spread

In [20]:
# Check for violations of vertical spread condition
def add_strike_violation_column(chunks, column_name, tol=0.0000):
    updated_chunks = []

    for chunk in chunks:

        C = chunk[f"{column_name}"].values

        violation_flags = np.zeros(len(chunk), dtype=bool)

        for i in range(1, len(C) - 1):
            if C[i] >= C[i-1] + tol:
                violation_flags[i] = True
        
        chunk[f"{column_name}_vertical_violation"] = violation_flags
        updated_chunks.append(chunk)

    return updated_chunks

for column_name in ["price", "call_price_nn", "call_price_hagan", "call_price_antonov"]:
    chunks = add_strike_violation_column(chunks, column_name)


In [21]:
df = pd.concat(chunks, ignore_index=True)

In [22]:
print(f"positivity nn: {len(df[df['call_price_nn_positivity_violation'] == True]) / len(df) * 100}")
print(f"positivity hagan: {len(df[df['call_price_hagan_positivity_violation'] == True]) / len(df) * 100}")
print(f"positivity antonov: {len(df[df['call_price_antonov_positivity_violation'] == True]) / len(df) * 100}")
print(f"positivity mc: {len(df[df['price_positivity_violation'] == True]) / len(df) * 100}")

positivity nn: 0.0
positivity hagan: 0.0
positivity antonov: 0.0
positivity mc: 0.0


In [23]:
print(f"vertical nn: {len(df[df['call_price_nn_vertical_violation'] == True]) / len(df) * 100}")
print(f"vertical hagan: {len(df[df['call_price_hagan_vertical_violation'] == True]) / len(df) * 100}")
print(f"vertical antonov: {len(df[df['call_price_antonov_vertical_violation'] == True]) / len(df) * 100}")
print(f"vertical mc: {len(df[df['price_vertical_violation'] == True]) / len(df) * 100}")

vertical nn: 0.08037342730841757
vertical hagan: 0.04018671365420878
vertical antonov: 0.06491699897987573
vertical mc: 0.0


In [24]:
# removing last chunk as it cannot have an arbitrage violation
chunks_no_last_strike = [chunk[:-1] for chunk in chunks] 
df2 = pd.concat(chunks_no_last_strike, ignore_index=True)

In [25]:
print(f"butterfly nn: {len(df2[df2['call_price_nn_butterfly_violation'] == True]) / len(df2) * 100}")
print(f"butterfly hagan: {len(df2[df2['call_price_hagan_butterfly_violation'] == True]) / len(df2) * 100}")
print(f"butterfly antonov: {len(df2[df2['call_price_antonov_butterfly_violation'] == True]) / len(df2) * 100}")
print(f"butterfly mc: {len(df2[df2['price_butterfly_violation'] == True]) / len(df2) * 100}")

butterfly nn: 2.8102969791699897
butterfly hagan: 0.7049666656033685
butterfly antonov: 0.5614214169510989
butterfly mc: 0.26795113081756994
